In [6]:
import numpy as np
from matplotlib import pyplot as plt
import scipy as sc
import random
import matplotlib
from scipy.stats import maxwell
import h5py

# Parameters
N = 1018
delgam = 0.1
me = 1
qe = -1
L = 1600

n = 10 # number of particles

ptcls = np.full(n, None) 

dt = 0.1 # timestep

c = 0.225 # TRISTAN-MP constant

a = np.sqrt(delgam)*c # value for Maxwell-Boltzmann sampling

frozent = 0.6 # time for frozen B field

simt = int(frozent*N/2) # time converted into array indices

tmax = 1000

t = np.arange(0, tmax, dt)

timeSteps = np.size(t)

#Ex = np.load(arr)[simt]

#Ey = np.load(arr)[simt]

#Ez = np.load(arr)[simt]

#Bx = np.load(arr)[simt]

#By = np.load(arr)[simt]

#Bz = np.load(arr)[simt]



In [2]:
class Particle:
    
    def __init__(self, m, L, q, a, timeSteps):
        self.m = m
        self.q = q
        
        vmag0 = maxwell.rvs(a)
        theta = 2 * np.pi * random.random()
        phi = np.pi * random.random()
        v0 = (vmag0*np.sin(theta)*np.cos(phi),vmag0*np.sin(theta)*np.sin(phi),vmag0*np.cos(theta))
        
        
        self.xp    = np.zeros(timeSteps)
        self.xp[0] = L/4 * random.random()
        self.yp    = np.zeros(timeSteps)
        self.yp[0] = L/4 * random.random()
        self.zp    = np.zeros(timeSteps)
        self.zp[0] = L/4 * random.random()
        self.vx    = np.zeros(timeSteps)  
        self.vx[0] = v0[0]
        self.vy    = np.zeros(timeSteps)
        self.vy[0] = v0[1]
        self.vz    = np.zeros(timeSteps)
        self.vz[0] = v0[2]
        
    def setx(self, x, i):
        self.xp[i] = x
        
    def sety(self, y, i):
        self.yp[i] = y
        
    def setz(self, z, i):
        self.zp[i] = z
        
    def setvx(self, vx, i):
        self.vx[i] = vx
        
    def setvy(self, vy, i):
        self.vy[i] = vy
        
    def setvz(self, vz, i):
        self.vz[i] = vz
        
    def getx(self, i):
        return self.xp[i]
        
    def gety(self, i):
        return self.yp[i]
        
    def getz(self, i):
        return self.zp[i]
        
    def getvx(self, i):
        return self.vx[i]
    
    def getvy(self, i):
        return self.vy[i]
        
    def getvz(self, i):
        return self.vz[i]
    
    def getm(self):    
        return self.m
    
    def getq(self):
        return self.q

In [3]:
def Boris(dt,x,y,z,vx,vy,vz,Ex,Ey,Ez,Bx,By,Bz,qi,c,mi):
#-----------------------------------------
#Propagates the position variables x,y,z and
#the velocity variables vx,vy,vz from t to t+dt
#for given E and B using Boris Method
#-----------------------------------------

    #Decompose E,B
    b0x = (qi*Bx*dt)/(2*c*mi)
    b0y = (qi*By*dt)/(2*c*mi)
    b0z = (qi*Bz*dt)/(2*c*mi)
    ex = Ex
    ey = Ey
    ez = Ez
    b0  = np.sqrt(b0x**2 + b0y**2 + b0z**2)
    
    #Half Electric Push
    v0x= vx  + (dt/2)*ex
    v0y= vy  + (dt/2)*ey
    v0z= vz  + (dt/2)*ez
    
    #Magnetic rotation
    ###(v0 + v0xb0)/(1 + b0²):
    v11x= (v0x + v0y*b0z - v0z*b0y)/(1 + b0**2)
    v11y= (v0y + v0z*b0x - v0x*b0z)/(1 + b0**2)
    v11z= (v0z + v0x*b0y - v0y*b0x)/(1 + b0**2)
    ###v0 + 2*v11xb0
    v1x= v0x + 2*(v11y*b0z - v11z*b0y)
    v1y= v0y + 2*(v11z*b0x - v11x*b0z)
    v1z= v0z + 2*(v11x*b0y - v11y*b0x)
    
    
    #Half Electric push
    vfx= v1x + (dt/2)*ex
    vfy= v1y + (dt/2)*ey
    vfz= v1z + (dt/2)*ez
    
    #Propagate position
    xf = x + vfx*dt
    yf = y + vfy*dt
    zf = z + vfz*dt
    
    return np.array([xf,vfx,yf,vfy,zf,vfz])
    

In [8]:
# Main Method
for i in range(n): # spawn particles
    ptcls[i] = Particle(me,L,qe, a, timeSteps)
    
for i in range(1, timeSteps): # Boris method loop
    for j in range(n):    
        x = ptcls[j].getx(i-1) # get particle positions
        y = ptcls[j].gety(i-1)
        z = ptcls[j].getz(i-1)
        vx = ptcls[j].getvx(i-1)
        vy = ptcls[j].getvy(i-1)
        vz = ptcls[j].getvz(i-1)
        
        m = ptcls[j].getm()
        q = ptcls[j].getq()
        
        #interpolate fields to particle positions
        i1 = int(x)
        i2 = int(y)
        
        boxSize = L/4

        deltax = x - i1
        deltay = y - i2
        deltaz = 0
        
        Bxxjk = ((1/2)*(Bx[i1][i2] + Bx[i1][i2-1]) 
                  + ((1/2)*(Bx[int((i1+1)%boxSize)][i2]+Bx[int((i1+1)%boxSize)][i2-1]) 
                     - (1/2)*(Bx[i1][i2] + Bx[i1][i2-1]))*deltax)
        
        Bxxj1k = ((1/2)*(Bx[i1][int((i2+1)%boxSize)] + Bx[i1][i2]) 
                  + ((1/2)*(Bx[int((i1+1)%boxSize)][int((i2+1)%boxSize)]+Bx[int((i1+1)%boxSize)][i2]) 
                     - (1/2)*(Bx[i1][int((i2+1)%boxSize)] + Bx[i1][i2]))*deltax)
        
        Bxptcl = Bxxjk + (Bxxj1k - Bxxjk)*deltay
        
        Byxjk = ((1/2)*(By[i1][i2] + By[i1][i2-1]) 
                  + ((1/2)*(By[int((i1+1)%boxSize)][i2]+By[int((i1+1)%boxSize)][i2-1]) 
                     - (1/2)*(By[i1][i2] + By[i1][i2-1]))*deltax)
        
        Byxj1k = ((1/2)*(By[i1][int((i2+1)%boxSize)] + By[i1][i2]) 
                  + ((1/2)*(By[int((i1+1)%boxSize)][int((i2+1)%boxSize)]+By[int((i1+1)%boxSize)][i2]) 
                     - (1/2)*(By[i1][int((i2+1)%boxSize)] + By[i1][i2]))*deltax) 
        
        Byptcl = Byxjk + (Byxj1k - Byxjk)*deltay
        
        Bzxjk = ((1/2)*(Bz[i1][i2] + Bz[i1][i2-1]) 
                  + ((1/2)*(Bz[int((i1+1)%boxSize)][i2]+Bz[int((i1+1)%boxSize)][i2-1]) 
                     - (1/2)*(Bz[i1][i2] + Bz[i1][i2-1]))*deltax)
        
        Bzxj1k = ((1/2)*(Bz[i1][int((i2+1)%boxSize)] + Bz[i1][i2]) 
                  + ((1/2)*(Bz[int((i1+1)%boxSize)][int((i2+1)%boxSize)]+Bz[int((i1+1)%boxSize)][i2]) 
                     - (1/2)*(Bz[i1][int((i2+1)%boxSize)] + Bz[i1][i2]))*deltax)
   
        Bzptcl = Bzxjk + (Bzxj1k - Bzxjk)*deltay
        
        Exxjk = (1/2)*(Ex[i1][i2] + Ex[i1-1][i2]) + (1/2)*(Ex[int((i1+1)%boxSize)][i2] + Ex[i1-1][i2])*deltax # Ex[x][j][k] interpolate over x
        
        Exxj1k = (1/2)*(Ex[i1][int((i2+1)%boxSize)] + Ex[i1-1][int((i2+1)%boxSize)]) + (1/2)*(Ex[int((i1+1)%boxSize)][int((i2+1)%boxSize)] + Ex[i1-1][int((i2+1)%boxSize)])*deltax # Ex[x][j+1][k]
            
        Exptcl = Exxjk + (Exxj1k - Exxjk)*deltay # Ex[x][y][k] perform interpolation over y
        
        Eyxjk = (1/2)*(Ey[i1][i2] + Ey[i1-1][i2]) + (1/2)*(Ey[int((i1+1)%boxSize)][i2] + Ey[i1-1][i2])*deltax # Ey[x][j][k] interpolate over x
        
        Eyxj1k = (1/2)*(Ey[i1][int((i2+1)%boxSize)] + Ey[i1-1][int((i2+1)%boxSize)]) + (1/2)*(Ey[int((i1+1)%boxSize)][int((i2+1)%boxSize)] + Ey[i1-1][int((i2+1)%boxSize)])*deltax # Ey[x][j+1][k]

        Eyptcl = Eyxjk + (Eyxj1k - Eyxjk)*deltay # Ey[x][y][k] perform interpolation over y
        
        Ezxjk = (1/2)*(Ez[i1][i2] + Ez[i1-1][i2]) + (1/2)*(Ez[int((i1+1)%boxSize)][i2] + Ez[i1-1][i2])*deltax # Ez[x][j][k] interpolate over x
        
        Ezxj1k = (1/2)*(Ez[i1][int((i2+1)%boxSize)] + Ez[i1-1][int((i2+1)%boxSize)]) + (1/2)*(Ez[int((i1+1)%boxSize)][int((i2+1)%boxSize)] + Ez[i1-1][int((i2+1)%boxSize)])*deltax # Ez[x][j+1][k]
        
        Ezptcl = Ezxjk + (Ezxj1k - Ezxjk)*deltay # Ez[x][y][k] perform interpolation over y
        
        ptcls[j].setx(Boris(dt,x,y,z,vx,vy,vz,Exptcl,Eyptcl,Ezptcl,Bxptcl,Byptcl,Bzptcl,q,c,m)[0]%(boxSize),i);
        ptcls[j].sety(Boris(dt,x,y,z,vx,vy,vz,Exptcl,Eyptcl,Ezptcl,Bxptcl,Byptcl,Bzptcl,q,c,m)[2]%(boxSize),i);
        ptcls[j].setz(Boris(dt,x,y,z,vx,vy,vz,Exptcl,Eyptcl,Ezptcl,Bxptcl,Byptcl,Bzptcl,q,c,m)[4]%(boxSize),i);
        ptcls[j].setvx(Boris(dt,x,y,z,vx,vy,vz,Exptcl,Eyptcl,Ezptcl,Bxptcl,Byptcl,Bzptcl,q,c,m)[1],i);
        ptcls[j].setvy(Boris(dt,x,y,z,vx,vy,vz,Exptcl,Eyptcl,Ezptcl,Bxptcl,Byptcl,Bzptcl,q,c,m)[3],i);
        ptcls[j].setvz(Boris(dt,x,y,z,vx,vy,vz,Exptcl,Eyptcl,Ezptcl,Bxptcl,Byptcl,Bzptcl,q,c,m)[5],i);